# Функции: избранные темы

## Вступление: контекстные менеджеры

В Python по умолчанию задана некоторая точность вычисления чисел с плавающей точкой, которая, что логично, может отличаться от требуемой для решения конкретной задачи точности. Для пример, будет рассматривать точность вычислений в библиотеке **decimal**.

Предположим, нас устраивает точность вычислений по умолчанию в этой библиотеке, но для некоторых расчетов мы бы хотели использовать повышенную или пониженную точность и только для них. Как нам быть? Тут-то нам на помощь и придут контекстные менеджеры.

В этом примере мы поговорим о контекстных менеджерах и напишем свой менеджер для установки точности вычислений, которые будет работать примерно так:

```python
with set_precision(5):
    ...
```

In [1]:
from decimal import Decimal, getcontext
from contextlib import contextmanager
from typing import Generator


@contextmanager
def set_precision(precision: int) -> Generator[None, None, None]:
    precision = int(precision)

    if precision < 0:
        raise ValueError("precision should be not negative integer")
    
    # сохранение контекста
    precision_old = getcontext().prec
    getcontext().prec = precision

    # возвращение управления
    try:
        yield
    
    # восстановвление контекста
    finally:
        getcontext().prec = precision_old
        


## Разминка: геометрическая прогрессия

Напишите бесконечный генератор геометрической прогрессии. В качестве параметров генератор должен принимать:  
- первый член прогрессии
- шаг прогрессии

In [2]:
class Body:
    def __init__(self) -> None: ...
    def leg(self) -> None: ...
    def kill(self) -> None: ...


class Body:
    def __init__(self) -> None:
        pass
    def leg(self) -> None:
        return 1
    def kill(self) -> None:
        return 0


b = Body()
b.kill()

0

## Задание 1: float_range

Необходимо создать аналог range(), который генерирует арифметическую прогрессию. Ниже приведены примеры использования:

*Пример 1*:

```python
for i in float_range(stop=5):
    print(i)
```

*Вывод*:
```console
0
1
2
3
4
```
___
*Пример 2*:

```python
for i in float_range(stop=1, step=0.5):
    print(i)
```

*Вывод*:
```console
0
0.5
```
___
*Пример 3*:

```python
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
```

*Вывод*:
```console
1
0.5
0
-0.5
```

In [95]:
from typing import Generator

def float_range(*, stop: float, start: float = 0, step: float = 1) -> Generator[float, None, None]:
    stop, start, step = map(float, locals().values())
    sign = step / abs(step)

    while sign*start < sign*stop:
        yield start
        start += step

In [98]:
for i in float_range(stop=7, start=10, step=-0.5):
    print(i)

10.0
9.5
9.0
8.5
8.0
7.5


## Задание 2: свой map

### Часть 1: копируем map

Реализуйте аналог функции map, полностью копирующий ее поведение. Саму map использовать нельзя.

In [26]:
from typing import Callable, Typevar, Generator, Iterable, Any


def rar(*iterables: Iterable[Any]) -> Generator[tuple[Any], None, None]:
    '''
    I prefer rar than zip
    ''' 

    mn_len = min(iterables, key=len)

    for i in range(len(mn_len)):
        res = tuple(iterable[i] for iterable in iterables)
        yield res


T = Typevar("T")
V = Typevar("V")
def karta_krest_krest(func: Callable[[T], V], *iterables: Iterable[T]) -> Generator[V, None, None]:
    '''
    What is map? It is karta_✝✝!
    '''

    for item in rar(*iterables):
        yield func(*item)

2

### Часть 2: дополняем map

Добавьте возможность управлять поведением вашего map'a: сделайте так, чтобы map имела возможность не только обрезать последовательности, но и дополнять короткие последовательности до динных. 

Совет: функция **zip_longest** из библиотеки **itertools** может оказаться полезной.

In [33]:
from enum import Enum
from itertools import zip_longest


class MapTypes(Enum):
    SHORTEST = 'short'
    LONGEST = 'long'


In [56]:
from typing import Callable, TypeVar, Generator, Iterable, Any


def rar_versta(*iterables: Iterable[Any], fill: bool = False, fill_value: Any = None) -> Generator[tuple[Any], None, None]:
    '''
    I prefer rar_versta than zip_longest
    ''' 

    if not iterables:
        raise ValueError("iterables must not be empty")

    i_len = len(max(iterables, key=len)) if fill else len(min(iterables, key=len))

    for i in range(i_len):
        res = tuple(iterable[i] if i < len(iterable) else fill_value
                    for iterable in iterables)
        yield res


T = TypeVar("T")
V = TypeVar("V")
def karta_rusi_krest_krest(func: Callable[[T], V], *iterables: Iterable[T],
                           type: MapTypes = MapTypes.SHORTEST, fill_value: T | None = None) -> Generator[V, None, None]:
    '''
    What is upgraded map? It is karta_rusi-✝✝!
    '''

    if not iterables:
        raise ValueError("iterables must not be empty")

    fill = type == MapTypes.LONGEST
    items = rar_versta(*iterables, fill=fill, fill_value=fill_value)

    for item in items:
        yield func(*item)

## Задание 3: Спиннер

### Часть 1: генератор

Напишите функцию, которая на вход получает коллекцию и возвращает генератор, последовательно возвращающий элементы коллекции, а после возврата последнего элемента коллекции очередной вызов генератора приведет к зацикливанию.


*Пример*:

```python
generator = generate_circle('abc')

print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
```
*Вывод*:
```console
a
b
c
a
b
```

In [1]:
from typing import Generator


def generate_circle(spiner_string: str = "abc") -> Generator[str, None, None]:
    spiner_string = str(spiner_string)

    if not spiner_string:
        raise ValueError("spiner string must not be empty")

    current_step = -1

    while True:
        current_step += 1
        if current_step == len(spiner_string):
            current_step = 0

        yield spiner_string[current_step]

### Часть 2: Колесо Сансары

Используя генератор из предыдущего раздела, реализуйте функцию, которая отображает на экране спиннер для индикации загрузки.

Отрисовка спиннера печатает на экран надпись: Thinking: \<symbol\>, где вместо \<symbol\> последовательно появляются знаки: \, |, /, -, что создаёт эффект вращения.

Вход функции: 
- time_limit - время (в секундах), в течение которого должна производиться отрисовка спиннера;
- pause - время (в секундах) задержки между сменой символов спиннера;

Интересная статья на тему индикаторов: https://dtf.ru/flood/174240-progress-bar-ili-spinner-chto-i-kogda-ispolzovat?ysclid=lorrg51syv550654720

In [2]:
import time


def load_water_from_Baikal(time_limit: float = 4.0, pause: float = 1.0) -> None:
    '''
    Baikal water is strongest weapon against lizards, but we could load it :(
    '''

    time_limit = float(time_limit)
    pause = float(pause)

    spiner_string = "|/-\\"

    gen = generate_circle(spiner_string)
    amount_itteration = int(time_limit/pause)
    message = "Water from Baikal is loading: "

    if amount_itteration < 1:
        time.sleep(pause)
    else:
        for _ in range(amount_itteration):
            time.sleep(pause)
            print(message, f"{next(gen)}", end="\r", flush=True)

In [ ]:
load_water_from_Baikal()